## OpenML model search pipelines
In this notebook, we analyze the scikit learn model search pipelines to include them in the graph database.

In [3]:
from openml import datasets, tasks, runs, flows, setups, config, evaluations
from workloadoptimization.essentials import Component, ExperimentObject, ExperimentGraph, ExperimentParser
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import uuid
import networkx as nx
import sklearn
from networkx.drawing.nx_agraph import graphviz_layout
from openmlstudy14.preprocessing import ConditionalImputer as openMLConditionalImputer
from hyperimp.utils.preprocessing import ConditionalImputer as hyperimpConditionalImputer

%matplotlib inline


config.apikey = '8e2079508005379adb9a60798696134e'
config.server = 'https://www.openml.org/api/v1'
config.set_cache_directory(os.path.expanduser('~/openml-cache'))

In [2]:
FLOW = 8365
TASK = 31

In [3]:
import pickle
def getTopRuns(numberOfRuns, pipeline, task):
    openMlEvaluations = evaluations.list_evaluations('predictive_accuracy', task= [task], flow = [pipeline])
    evaluationData = pd.DataFrame.from_dict(openMlEvaluations, orient='index')
    evaluationData['accuracy'] = evaluationData.apply(lambda eva: eva.values[0].value, axis = 1)
    evaluationData['run_id'] = evaluationData.apply(lambda eva: eva.values[0].run_id, axis = 1)
    # extracting the top ''numberOfRuns' runs
    topRuns = evaluationData.sort_values('accuracy',ascending=False)
    if (numberOfRuns>0):
        topRuns = topRuns[0:numberOfRuns]
    # retreiving the run objects from the top runs
    openMLRuns = runs.list_runs(task=[task], flow=[pipeline])
    experiments = pd.DataFrame.from_dict(openMLRuns,orient='index')
    Experiment = experiments.merge(topRuns,on='run_id').drop(columns=['uploader',0])
    # TODO: This is the limit on the api size, I should fix it to make calls in batches
    Setup = pd.DataFrame.from_dict(setups.list_setups(setup=Experiment.setup_id[0:500],size = 500 ), orient='index').reset_index()
    Setup.columns=['id', 'setup']

    return pd.merge(Setup, Experiment, how = 'inner', left_on='id', right_on='setup_id').drop(columns = ['id','setup_id'])[['run_id','task_id','flow_id', 'accuracy','setup']]
def extractExperiments(filePath, taskIds, flowIds):
    if os.path.isfile(filePath):
        return pd.read_pickle(filePath)
    frames = []
    for t in taskIds:
        for f in flowIds:
            frames.append(getTopRuns(100000,f,t))
    Experiments = pd.concat(frames).reset_index(drop=True)
    Experiments.to_pickle(filePath)
    return Experiments

# This is time consuming, so it is better to persist the list of the runs to disk
# If you are changing the tasks or flow ids, remember to change the name of the file 
#Experiments = extractExperiments('meta/massive-study-picked-experiment-task-31', TASK_IDS, FLOW_IDS)


In [36]:
#openMLRuns = runs.list_runs(task=[TASK], flow=[FLOW], size = 10)
# openMLRuns{'flow_id': 8365,'run_id': 9191572, 'setup_id': 7122364, 'task_id': 31,  'uploader': 1935}
s = setups.get_setup(7122509)

In [4]:
for k,v in s.parameters.iteritems():
    print v.full_name,v.value

NameError: name 's' is not defined

In [4]:
flow = flows.get_flow(FLOW)
skPipeline = flows.flow_to_sklearn(flow)

In [5]:
skPipeline

RandomizedSearchCV(cv=3, error_score=u'raise',
          estimator=Pipeline(memory=None,
     steps=[(u'imputation', ConditionalImputer(axis=0,
          categorical_features=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
          copy=True, fill_empty=0, missing_values=u'NaN', strategy=u'mean',
       ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid=True, n_iter=50, n_jobs=-1,
          param_distributions=OrderedDict([(u'clf__bootstrap', [True, False]), (u'clf__criterion', [u'gini', u'entropy']), (u'clf__max_features', [0.05685145935354432]), (u'clf__min_samples_leaf', <scipy.stats._distn_infrastructure.rv_frozen object at 0x115c0de50>), (u'clf__min_samples_split', <scipy.stats._distn_infrastructure.rv_frozen object at 0x115c19350>), (u'clf__n_estimators', [300]), (u'clf__random_state', <scipy.stats._distn_in

In [6]:
type('asd')

str